## Imports, Read data, and Generate Model

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import requests, json

df = pd.read_csv("./SalaryData.csv")

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

df_copy = train_set.copy()

test_set_full = test_set.copy()
test_set = test_set.drop(["Salary"], axis=1)

train_labels = df_copy["Salary"]

train_set_full = train_set.copy()
train_set = train_set.drop(["Salary"], axis=1)

lin_reg = LinearRegression()

lin_reg.fit(train_set, train_labels)

salary_pred = lin_reg.predict(test_set)

salary_pred

C:\Users\z023208\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\z023208\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\z023208\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\z023208\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


array([115790.21011287,  71498.27809463, 102596.86866063,  75267.80422384,
        55477.79204548,  60189.69970699])

In [7]:
lin_reg.predict([[10.0]])

array([119559.73624209])

In [8]:
lin_reg

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Run Model API

### Persist Model and Data

#### Python Pickle

In [1]:
import pickle

In [2]:
from sklearn.externals import joblib

In [5]:
import dill

In [4]:
!pip install dill

  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78598 sha256=7b450020e7f30828a8931c7e9794fdec9939d30d79179c290c84b699be5ecbc3
  Stored in directory: C:\Users\z023208\AppData\Local\pip\Cache\wheels\59\b1\91\f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
Successfully built dill


In [10]:
import pickle

with open("python_lin_reg_model.pkl", "wb") as file_handler:
    pickle.dump(lin_reg, file_handler)
    
with open("python_lin_reg_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
joblib.dump(lin_reg,"model.pkl")

['model.pkl']

In [13]:
model = joblib.load('model.pkl')

In [14]:
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
model.predict([[12]])

array([138407.36688815])

#### `scikit-learn`'s `joblib`

In [ ]:
Running on http://127.0.0.1:9080/ (Press CTRL+C to quit)

In [16]:
BASE_URL = "http://localhost:9080"

joblib.dump(lin_reg, "linear_regression_model.pkl")

joblib.dump(train_set, "training_data.pkl")
joblib.dump(train_labels, "training_labels.pkl")

['training_labels.pkl']

### Predict API

In [36]:
years_exp = {"yearsOfExperience": 30}

response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

#response

In [37]:
response

<Response [200]>

In [76]:
a = response.content

In [77]:
a = response.content.decode('utf-8')

In [78]:
type(a)

str

In [79]:
a

'[273913.95469281997]'

In [80]:
a = a[1:-1]

In [81]:
float(a)

273913.95469281997

In [20]:
df_copy.query('YearsExperience > 7 and YearsExperience <= 8')

,YearsExperience,Salary
22,7.9,101302.0
21,7.1,98273.0


### Retrain API

In [21]:
data = json.dumps([{"YearsExperience": 40,"Salary": 140000}, 
                   {"YearsExperience": 12.1,"Salary": 142000},
                  {"YearsExperience": 16.1,"Salary": 162000},
                  {"YearsExperience": 24.1,"Salary": 242000},
                  {"YearsExperience": 35.1,"Salary": 542000}])

data

'[{"YearsExperience": 40, "Salary": 140000}, {"YearsExperience": 12.1, "Salary": 142000}, {"YearsExperience": 16.1, "Salary": 162000}, {"YearsExperience": 24.1, "Salary": 242000}, {"YearsExperience": 35.1, "Salary": 542000}]'

In [22]:
response = requests.post("{}/retrain".format(BASE_URL), json = data)

response.json()

{'data': 'Retrained model successfully.'}

In [56]:
response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

response.json()

[195734.81688189803]

### Current Details API

In [23]:
response = requests.get("{}/currentDetails".format(BASE_URL))

response.json()

{'coefficients': [7983.3711148548455],
 'intercepts': 34412.82124717464,
 'score': 0.6147668201487855}